
# Feed Forward 
__1. All nodes are fully connected.__<br>
__2. Activation flows from input layer to output, without back loops.__ <Br>
__3. There is one layer between input.__<Br>
__4. [Towards Data Science Blog BY Andrew Tch for neural networks details ](https://towardsdatascience.com/the-mostly-complete-chart-of-neural-networks-explained-3fb6f2367464).__


### Titanic Dataset is used to implement Feed Forward
[Download Titanic Dataset](https://www.kaggle.com/c/titanic/data).


In [1]:
import numpy as np 
import pandas as pd


In [2]:
def reading_test_train_data():
    train = pd.read_csv('./datasets/train.csv')
    test = pd.read_csv('./datasets/test.csv')
    return train,test

In [3]:
train,test = reading_test_train_data()


In [37]:
# first 10 rows
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,28.0,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [6]:
# 891 rows 
# 12 columns /featurs including target
train.shape

(891, 12)

In [34]:
# only age count() is less then total df len means contains nan values
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [36]:
# filling age with age median
train['Age'] = train['Age'].fillna(train['Age'].median())

In [43]:
# cabin contians a lot of nan so it is better to drop this column
len(train[train['Cabin'].isnull()])

687

In [47]:
# id and ticket logically are not usefull 
train.drop(['PassengerId','Ticket','Cabin'],axis=1,inplace=True)